# Library Scanner Testing

Test the media scanner with realistic file structures.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import shutil
from pathlib import Path
from unittest.mock import patch

from alma_tv.library.scanner import Scanner
from alma_tv.database import get_db, Video, init_db


## Setup Test Media Directory

In [ ]:
# Create temp directory
temp_media = Path("temp_test_media")
if temp_media.exists():
    shutil.rmtree(temp_media)
temp_media.mkdir()

# Create realistic file structure
(temp_media / "Bluey").mkdir()
(temp_media / "Bluey" / "Bluey_S01E01_MagicXylophone.mp4").touch()
(temp_media / "Bluey" / "Bluey_S01E02_Hospital.mp4").touch()
(temp_media / "Bluey" / "Bluey_S02E01_DanceMode.mp4").touch()

(temp_media / "Peppa_Pig").mkdir()
(temp_media / "Peppa_Pig" / "PeppaPig_S01E01_MuddyPuddles.mp4").touch()
(temp_media / "Peppa_Pig" / "PeppaPig_S01E02_MrDinosaurIsLost.mp4").touch()

print(f"Created test media in: {temp_media.absolute()}")


## Scan Directory

In [ ]:
scanner = Scanner(media_root=temp_media)

# Mock ffprobe to avoid dependency on actual media files
with patch.object(scanner, 'get_duration', return_value=420):
    summary = scanner.scan_directory()

print(f"Scan Summary: {summary}")


## Verify Database

In [ ]:
with get_db() as db:
    videos = db.query(Video).all()
    print(f"Total videos in DB: {len(videos)}")
    for v in videos:
        print(f"  {v.series} {v.episode_code} - {v.duration_seconds}s")


## Test Incremental Scan

In [ ]:
# Add a new file
(temp_media / "Bluey" / "Bluey_S02E02_Hammerbarn.mp4").touch()

# Rescan
with patch.object(scanner, 'get_duration', return_value=430):
    summary = scanner.scan_directory()

print(f"Incremental Scan: {summary}")


## Cleanup

In [ ]:
# shutil.rmtree(temp_media)